<div style="background-color:black;color:white;padding:20px">

<center><h3> LAA </h3></center>
<center><h1> Projeto Bicóptero </h1></center>

<center> <h1> Modelo dinâmico do conjunto braço robótico-bicóptero usando a formulação de Lagrange</h1></center>

<h3> Por Gabriel Henrique Costa e Silva </h3> 
</div>

<h2> Inicializando as bibliotecas </h2>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import * 
from sympy import trigsimp, pi, cos, sin
from sympy.physics.mechanics import *

In [2]:
from __future__ import print_function, division
from sympy import symbols, simplify
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point

In [3]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

In [4]:
from IPython.display import Image

In [5]:
t = Symbol("t")

# Angulos Theta
theta1 = Function("theta1")(t)
theta2 = Function("theta2")(t)
theta3 = Function("theta3")(t)
theta4 = Function("theta4")(t)

# Constante da gravidade
g = dynamicsymbols('g')

# Forcas e Momentos
fx, fy, fz = dynamicsymbols('f_x, f_y, f_z')
taux, tauy, tauz = dynamicsymbols('tau_x, tau_y, tau_z')

# Forcas dos motores
f1,f2= dynamicsymbols('F_1, F_2')

# Angulos Alfa (Angulos do mecanismo tilt)
alpha1 = Function("alpha1")(t)
alpha2 = Function("alpha2")(t)

# Extensão dos bracos do bicopter
b,b1,b2 = dynamicsymbols('b, b_1, b_2')

# Extensão dos segmentos do braço robótico
l1, l2, l3, l4 = dynamicsymbols('l1, l2, l3, l4')
lg1, lg2, lg3, lg4 = dynamicsymbols('l_{G_1}, l_{G_2}, l_{G_3}, l_{G_4}')

#Características inerciais
m1, m2, m3, m4, mp = dynamicsymbols('m_1, m_2, m_3, m_4, m_p')
Ix1, Ix2, Ix3, Ix4, Ixp = dynamicsymbols('Ix_1, Ix_2, Ix_3, Ix_4, Ix_p')
Iy1, Iy2, Iy3, Iy4, Iyp = dynamicsymbols('Iy_1, Iy_2, Iy_3, Iy_4, Iy_p')
Iz1, Iz2, Iz3, Iz4, Izp = dynamicsymbols('Iz_1, Iz_2, Iz_3, Iz_4, Iz_p')
vx, vy, vz, wx, wy, wz = dynamicsymbols('v_x, v_y, v_z, omega_x, omega_y, omega_z')
# Ganho de atrito ?
fs1, fs2, fs3, fs4, fs5 = dynamicsymbols('fs1, fs2, fs3, fs4, fs5')

ang = Function("beta")(t)
# Variaveis no estado de equilibrio

#estados
theta10, theta20, theta30, theta40 = dynamicsymbols('theta_1_0, theta2_0, theta3_0, theta4_0')
omega10, omega20, omega30, omega40 = dynamicsymbols('omega1_0, omega2_0, omega3_0, omega4_0')
psi10, psi20, psi30, psi40 = dynamicsymbols('psi1_0, psi2_0, psi3_0, psi4_0')

# comandos
alpha10, alpha20= dynamicsymbols('alpha_1_0, alpha2_0')
f10, f20= dynamicsymbols('F_10, F_20')

# simplificacoes
num1, num2, num3, num4 = dynamicsymbols('num1, num2, num3, num4')
den1, den2, den3, den4 = dynamicsymbols('den1, den2, den3, den4')

In [6]:
# Criando a função de criação da matriz de transformação homogênea

# Matriz Rotação da Transformação
def rt_M(T):
    M = T[0:3,0:3]
    return M

In [7]:
# Matriz Rotação da Transformação
def ch_M(T):
    M=zeros(6)
    M[0:3,0:3] = T[0:3,0:3]
    M[3:6,3:6] = T[0:3,0:3]
    return M

In [8]:
# Matriz da massa e inércia
def mI_M(m,Ix,Iy,Iz):
    M = diag(m, m, m, Ix, Iy, Iz)
    return M

In [9]:
# Ponto de representação homogenea para dimensão 3
def ph_2N(ph):
    p = Matrix(ph[0:3])
    return (p)

In [10]:
# Definindo a matriz jacobiana
def Jc(j1,j2,j3,j4,j5):
    J = zeros(6,1)
    J = J.col_insert(0,j1)
    J = J.col_insert(1,j2)
    J = J.col_insert(2,j3)
    J = J.col_insert(3,j4)
    J = J.col_insert(4,j5)
    J.col_del(5)
    return J

In [11]:
# Definindo a nesima coluna da matriz jacobiana
def Ji(m0,op,oi):
    aux1 = m0*Matrix([0,0,1])
    aux2 = aux1.cross(op-oi)
    J = Matrix([aux2, aux1])
    return J

In [12]:
# Definindo a nesima coluna da matriz jacobiana
def JpTi(m0,op,Ti):
    # Calculate the zi= i-1Mi.k
    aux1 = m0*Matrix([0,0,1])
    oi = Matrix([Ti[0:3,3]])
    # Calculate the zi= i-1Mi.k
    aux2 = aux1.cross(op-oi)
    
    J = Matrix([aux2, aux1])
    return J

In [13]:
#  Matriz que vezes o q_i é igual o torsor cinemático que é na verdade uma linha do jacobiano cinemático
def v_i(M0,Tp,Ti): 
    # Calculate the zi= i-1Mi.k
    aux1 = M0*Matrix([0,0,1])

    Op = Tp[0:3,3]
    Oi = Ti[0:3,3]

    # Cross Product  zi x (Op-Oi)
    aux2 = (aux1.cross(Op-Oi))
    
    # for pivot joints
    
    v = Matrix([aux2, aux1])
    return v 

In [14]:
# Criando a função de criação da matriz de transformação homogênea
# a partir dos parâmetros de Denavit-Hatemberg

#cos = cos
#sin = sin

# Matriz Transformação Homogênea (Homogeous Transformation Matrix)
# Using the Standard Denavit-Hatemberg Convention (not Modified)
def ht_M(theta, d, alpha, a):
    # theta e alpha em radianos
    # d e a em metros
    Rz = Matrix([[cos(theta), -sin(theta), 0, 0],
                   [sin(theta), cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, a],
                   [0, 0, 0, 1]])
    ta = Matrix([[1, 0, 0, d],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = Matrix([[1, 0, 0, 0],
                   [0, cos(alpha), -sin(alpha), 0],
                   [0, sin(alpha), cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

<h2> Declaração dos parâmetros de Denavit-Hatenberg </h2>
<h2> Instanciação das matrizes de transformação homogêneas </h2>

<div>
<img src="img/modelo.png" width="500">
</div>


|  Elemento | $\alpha_i$ |  $d_i$   | $\theta_i$ |  $r_i$ |
| -------- | -------- | --------| -------- | -------- | 
| $1$ | $\frac{\pi}{2}$   | 0     | $\theta_1$ | $l_1$ |
| $2$ | 0                 | $l_2$ | $\theta_2$ | 0     |
| $3$ | $- \frac{\pi}{2}$ | $l_3$ | $\theta_3$ | 0     |
| $4$ | 0                 | $l_4$ | $\theta_4$ | 0     |

<h2> Relação entre $R_4$ e $R_P$ </h2>

<div>
<img src="img/referencialterminal.png" width="400">
</div>

Em particular, a matriz de transformação de $R_p$ para $R_4$ representa somente uma rotação de $\frac{\pi}{2}$~rad em torno de $Y_{j4}$, isto é,

\begin{equation}
    {}^{4}T_{P} = \begin{bmatrix}
    \cos{\frac{\pi}{2}} & 0 & \sin{\frac{\pi}{2}} & 0 \\
    0 & 1 & 0 & 0\\
    -\sin{\frac{\pi}{2}} & 0 & \cos{\frac{\pi}{2}} & 0 \\
    0 & 0 & 0 & 1 
    \end{bmatrix}=
    \begin{bmatrix}
    0 & 0 & 1 & 0 \\
    0 & 1 & 0 & 0\\
    -1 & 0 & 0 & 0 \\
    0 & 0 & 0 & 1 
    \end{bmatrix}
    \label{eq:T_P_4}
\end{equation}

\begin{equation}
{}^{0}T_{P} = {}^{0}T_{4}{}^{4}T_{P}\label{eq:T_P_0} 
\end{equation}

A matriz ${}^{0}T_{P}$ define a rotação e a posição CG do bicóptero em relação ao referencial $R_0$.

In [15]:
T1 = ht_M(theta1, 0, (pi)/2, l1)
T2 = ht_M(theta2, l2, 0, 0)
T3 = ht_M(theta3, l3, (-pi)/2, 0)
T4 = ht_M(theta4, l4, 0, 0)
ang=pi/2
TP = Matrix([[cos(ang), 0, sin(ang), 0],[0, 1, 0, 0],[-sin(ang), 0, cos(ang), 0],[0, 0, 0, 1]])

In [16]:
#T1 
#T2
#T3
#T4
#TP

<h3> Simplificação da matriz T3 a partir da relação entre $\theta_3$ e $\theta_2$</h3>

É importante ressaltar que há a seguinte relação algébrica entre $\theta_3$ e $\theta_2$:

\begin{equation}
\theta_3 = -\dfrac{\pi}{2} - \theta_2
\label{eq:relacaotheta2theta3}
\end{equation}

pois o elemento 2 está sempre alinhado com $Z_g$ devido à força peso.

<div>
<img src="img/relation_thetas.png" width="400">
</div>

In [17]:
T3

Matrix([
[cos(theta3),  0, -sin(theta3), l3*cos(theta3)],
[sin(theta3),  0,  cos(theta3), l3*sin(theta3)],
[          0, -1,            0,              0],
[          0,  0,            0,              1]])

In [18]:
T3=T3.subs(theta3,(-pi/2-theta2))

In [19]:
T3

Matrix([
[-sin(theta2),  0,  cos(theta2), -l3*sin(theta2)],
[-cos(theta2),  0, -sin(theta2), -l3*cos(theta2)],
[           0, -1,            0,               0],
[           0,  0,            0,               1]])

<h2> Instanciação das matrizes de transformação homogêneas em relação ao referencial 0 </h2>

In [20]:
T01=T1
T02=T01*T2
T02=trigsimp(T02)
T03=T02*T3
T03=trigsimp(T03)
T04=T03*T4
T0P=T04*TP

In [21]:
#(T01)
#(T02).subs({theta1:0,theta2:0})
#(T03)
#(T04)

<h1> Definição das variáveis articulares </h1>
Acrescenta-se o termo 1 para adicionar o bicoptero ao modelo, ele não se caracteriza como uma junta por isso a constante 1;

\begin{align}
    q &= \begin{bmatrix}
    \theta_1 \\
    \theta_2 \\
    \theta_3 \\
    \theta_4 \\
    1
    \end{bmatrix} &
    \dot{q} &= \begin{bmatrix}
    \dot{\theta_1} \\
    \dot{\theta_2} \\
    \dot{\theta_3} \\
    \dot{\theta_4}\\
    0\end{bmatrix} &
    \ddot{q} &= \begin{bmatrix}
    \ddot{\theta_1} \\
    \ddot{\theta_2} \\
    \ddot{\theta_3} \\
    \ddot{\theta_4}\\
    0\end{bmatrix}
\end{align}

In [22]:
q=Matrix([theta1,theta2,theta3,theta4,1])
q_dot= diff(q,t)
q_ddot= diff(q_dot,t)

<h2> Criação das matrizes Jacobianas cinemáticas</h2>

Ao contrário do modelo geométrico que corresponde à uma relação não linear entre \textbf{x} e \textbf{q}, o modelo cinemático corresponde a uma relação linear entre $\dot{x}$ e $\dot{q}$. O modelo cinemático pode ser escrito da seguinte maneira:

\begin{equation}
    \dot{x}=J(q) \cdot \dot{q}
\end{equation}

A matriz J(q) é nomeada matriz jacobiana ou simplesmente a jacobiana do robô. Esta matriz é de dimensão ($d\times n$), onde $d$ é o número de graus de liberdade do robô (no máximo $d=6$) e $n$ o número de eixos.

Por convenção, as matrizes jacobianas apresentadas neste documento serão sempre expressas no referencial $R_0$.

No caso do braço robótico da bancada por ser constituído apenas por juntas de revolução, a  coluna  i  da  Matriz Jacobiano do manipulador  é  dada  pela  seguinte  expressão, considerando ainda n índice referente ao referencial do efetuador:

\begin{equation}
    \textbf{J}_i=
    \begin{bmatrix}
    z_{i-1}\times \overrightarrow{O_{i-1}O_n} \\
    z_{i-1}
    \end{bmatrix}
    \label{eq:jc}
\end{equation}

De maneira mais objetiva $z_{j}$ corresponde ao vetor formado pelas três primeiros elementos da terceira coluna da matriz transformação homogênea ${}^{0}T_{j}$. E $O_j$ corresponde aos três primeiros elementos da quarta coluna de ${}^{0}T_{j}$. Assim, em nosso caso faz-se necessário apenas o cálculo das matrizes ${}^{0}T_{j}$, j=1,2,3,4 para se calcular a matriz jacobiana cinemática do efetuador (órgão final).



<div>
<img src="img/centro_gravidade.png" width="800">
</div>


\begin{align}
    O_{G_1}&=\begin{bmatrix} 0   \\ 0 \\ l_{G_1}       \end{bmatrix} {}&\text{, }
    O_{G_2}&=\begin{bmatrix} 0   \\ 0 \\ l_1          \end{bmatrix} {}&\text{, }
    O_{G_3}&=\begin{bmatrix} l_2 \\ 0 \\ l_1 - l_{G_3} \end{bmatrix} {}&\text{, }
    O_{G_4}&=\begin{bmatrix} l_2 \\ 0 \\ l_1 - l_3     \end{bmatrix} {}&\text{e }
    O_{G_p}&=\begin{bmatrix} l_2 \\ 0 \\ l_1 - l_3 - l_4 \end{bmatrix} {}&\text{.}
\end{align}


|  Elemento | $l_{i}$ [m]  |  $l_{G_i}$ [m]  |
| --- | --- | --- | 
| 1 | 0.3250 | 0.1414 |
| 2 | 0.5950 | 0.0000 | 
| 3 | 0.1730 | 0.0516 | 
| 4 | 0.2435 | 0.0000 | 
| P | 0.0000 | 0.0000 |

|  Elemento |  Massa [$kg$]  |  <td colspan=3> M. de Inércia [$kg.m^2$] |  |  |
| --- | --- | --- | --- | --- |
|  |  | Ixx | Iyy | Izz |
1  | 2,325   | 0,102  | 0,001 | 0,102 |
2  | 2,761 | 0,004  | 0,288  | 0,290 |
3  | 0,154 | 0,001 | 1,835e-5 | 0,001 |
4 | 0,115  | 1,723e-5 | 7,608e-5 | 8,260e-5 |
p | 0,565 | 0,014 | 5,000e-4 | 0,013 |
Total | 5,915  | 0,141 | 0,585  | 0,699 |

In [23]:
# Criação dos pontos origens em relaçao ao referencial R_0 pertencente ao dominio R^1x4
#perceba que og3 esta definido a partir do referencial R_2 por isso usa-se T02
#perceba que og2 esta na origem de R_1 
#perceba que og4 esta na origem de R_3 
#perceba que ogp esta na origem de R_p 
cg_origem={lg2:0,lg4:0}
og1=Matrix([0,0,lg1]).subs(cg_origem)
og2=ph_2N(T01*Matrix([lg2,0,0,1])).subs(cg_origem)
og3=ph_2N(T02*Matrix([0,-lg3,0,1])).subs(cg_origem)
og4=ph_2N(T03*Matrix([lg4,0,0,1])).subs(cg_origem)
ogp=ph_2N(T0P*Matrix([0,0,0,1])).subs(cg_origem)

In [143]:
#og1.subs({theta1:0,theta2:0,theta4:0})
#og2.subs({theta1:0,theta2:0,theta4:0})
#og3.subs({theta1:0,theta2:0,theta4:0})
#og4.subs({theta1:0,theta2:0,theta4:0})
#ogp.subs({theta1:0,theta2:0,theta4:0})

In [25]:
# Calculo do Jc[1] que é o jacobiano referente ponto centro de gravidade g1
null=Matrix([0,0,0,0,0,0])
J1CG1 = Ji(eye(3),og1, Matrix([0,0,0])) 
Jc1 = Jc(J1CG1,null,null,null,null)
# Calculo do Jc[2] que é o jacobiano referente ponto centro de gravidade g2
J1CG2= Ji(eye(3),og2, Matrix([0,0,0]))
J2CG2 =JpTi(rt_M(T01),og2,T01)
Jc2 = trigsimp(Jc(J1CG2,J2CG2,null,null,null))
# Calculo do Jc[3] que é o jacobiano referente ponto centro de gravidade g3
J1CG3 = Ji(eye(3),og3, Matrix([0,0,0])) 
J2CG3 = JpTi(rt_M(T01),og3,T01)
J3CG3 = JpTi(rt_M(T02),og3,T02) 
Jc3 = trigsimp(Jc(J1CG3,J2CG3,J3CG3,null,null))
# Calculo do Jc[4] que é o jacobiano referente ponto centro de gravidade g4
J1CG4 = Ji(eye(3),og4, Matrix([0,0,0])) 
J2CG4 = trigsimp(JpTi(rt_M(T01),og4, T01))
J3CG4 = JpTi(rt_M(T02),og4,T02) 
J4CG4 = JpTi(rt_M(T03),og4,T03)
Jc4 = trigsimp(Jc(J1CG4,J2CG4,J3CG4,J4CG4,null))
# Calculo do Jc[5] que é o jacobiano referente ponto centro de gravidade g5
J1CGp = Ji(eye(3),ogp, Matrix([0,0,0])) 
J2CGp = trigsimp(JpTi(rt_M(T01),ogp, T01))
J3CGp = JpTi(rt_M(T02),ogp,T02) 
J4CGp = JpTi(rt_M(T03),ogp,T03)
JpCGp = JpTi(rt_M(T03),ogp,T04)
Jcp = trigsimp(Jc(J1CGp,J2CGp,J3CGp,J4CGp,JpCGp))

In [26]:
# tensor cinematico velocidades lineares e angulares concatenados
#Matrix([vx, vy, vz, wx, wy, wz])

trim_position={theta1:theta10,theta2:theta20,theta3:theta30,theta4:theta40}
trim_speed={diff(theta10,t):omega10,diff(theta20,t):omega20,diff(theta30,t):omega30,diff(theta40,t):omega40}
trim_acel={diff(diff(theta10,t),t):psi10,diff(diff(theta20,t),t):psi20,diff(diff(theta30,t),t):psi30,diff(diff(theta40,t),t):psi40}

#Estados iniciais
val_pos={theta10: 0.00,theta20: 0.00,theta30: -(pi/2),theta40: 0.00}

#(Jc1*q_dot).subs(trim_position).subs(trim_speed).subs(val_pos)
#(Jc2*q_dot).subs(trim_position).subs(trim_speed).subs(val_pos)
#(Jc3*q_dot).subs(trim_position).subs(trim_speed).subs(val_pos)
#(Jc4*q_dot).subs(trim_position).subs(trim_speed).subs(val_pos)
#(Jcp*q_dot).subs(trim_position).subs(trim_speed).subs(val_pos)

In [27]:
# Calculo da matriz jacobiana cinematica aplicada no centro de gravidade do elemento p
J = trigsimp(Jc(J1CGp,J2CGp,J3CGp,J4CGp,JpCGp))
J

Matrix([
[-l2*sin(theta1)*cos(theta2) - l4*sin(theta4)*cos(theta1), -(l2*sin(theta2) - l3 - l4*cos(theta4))*cos(theta1), (l3 + l4*cos(theta4))*cos(theta1), -l4*sin(theta1)*cos(theta4),           0],
[ l2*cos(theta1)*cos(theta2) - l4*sin(theta1)*sin(theta4), -(l2*sin(theta2) - l3 - l4*cos(theta4))*sin(theta1), (l3 + l4*cos(theta4))*sin(theta1),  l4*cos(theta1)*cos(theta4),           0],
[                                                       0,                                      l2*cos(theta2),                                 0,              l4*sin(theta4),           0],
[                                                       0,                                         sin(theta1),                       sin(theta1),                 cos(theta1), cos(theta1)],
[                                                       0,                                        -cos(theta1),                      -cos(theta1),                 sin(theta1), sin(theta1)],
[                                        

<h1> Calculo da matriz jacobiana cinemática a partir do torsor cinematico</h1>

Outra maneira de calcular a matriz jacobiana cinemática é por meio do torsor cinemático.

O torsor cinemático é o vetor das velocidades lineares e angulares concatenados 

$\begin{equation}
{}^{i-1}V_{i/i-1}(P)=\begin{bmatrix}
v_x\\
v_y\\
v_z\\
w_x\\
w_y\\
w_z
\end{bmatrix}
\end{equation}$

Para a junta de revolução tem-se

$\begin{equation}
{}^{i-1}V_{i/i-1}(P)
=\begin{bmatrix} \vec{z_i}\cdot{}^{i-1}M_i\overrightarrow{O_iP_{/R_i}}
\\\vec{z_i}\end{bmatrix}
\vec{q}_i
\end{equation}$

em que 
$\begin{equation}
\begin{bmatrix} \vec{z_i}\cdot{}^{i-1}M_i\overrightarrow{O_iP_{/R_i}}
\\\vec{z_i}\end{bmatrix}
\end{equation}$ chamada abaixo de $v_i$ representa uma coluna da matriz jacobiana

In [28]:
J1 = v_i(eye(3), T0P, zeros(4,4))

J2 = trigsimp(v_i(rt_M(T01), T0P, T01))

J3 = v_i(rt_M(T02),T0P,T02)

J4 = v_i(rt_M(T03),T0P,T03)

JP = v_i(rt_M(T04),T0P,T04)

JcPP = trigsimp(Jc(J1,J2,J3,J4,JP))

# Se quiser calcular a velocidade a partir das velocidades das juntas
#v = (JcP4*Matrix([0,1,-1,0])).subs({theta1: 0, theta2:0,theta4: 0})

In [29]:
J-JcPP #São iguais os dois jeitos graças a Deus

Matrix([
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0],
[0, 0, 0, 0, 0]])

<h2> Calculo da Energia Cinética </h2>

\begin{equation}
    \mathcal{K}  = \frac{1}{2}\dot{\textbf{q}}^T\textbf{D}(\textbf{q})\dot{\textbf{q}}
    \label{eq:kineticEnergy_sum_concise}
\end{equation}

Sendo $\textbf{D}(\textbf{q})=\sum_{i=1}^n\textbf{D}_i(\textbf{q})$.


<h3> Criação das matrizes de Inércia</h3>

In [30]:
# a massa 4 diz respeito ao conjunto junta de theta4 e bicoptero
mI_1 = mI_M(m1,Ix1,Iy1,Iz1)
mI_2 = mI_M(m2,Ix2,Iy2,Iz2)
mI_3 = mI_M(m3,Ix3,Iy3,Iz3)
mI_4 = mI_M(m4,Ix4,Iy4,Iz4)
mI_p = mI_M(mp,Ixp,Iyp,Izp)
#mI_4

<h3> Criação de $D(q)$</h3>

\begin{equation}
    D(q)= \sum_{i=1}^5[J(G_i,q)]^T\cdot\begin{bmatrix}
    m_i & 0 & 0 & \emptyset_{1x3}\\
    0 & m_i & 0 & \emptyset_{1x3}\\
    0 & 0 & m_i & \emptyset_{1x3}\\
    \emptyset_{3x3} & \emptyset_{3x3} & \emptyset_{3x3} & \bar{I}_i
    \end{bmatrix}\cdot J(G_i,q)
     \label{eq:D_euler_lagrange}
\end{equation}

sendo que $G_i$ o centro de gravidade do elemento $i$, $\emptyset_{3x3}$ uma matriz de zeros, $\emptyset_{1x3}$ um vetor linha de zeros, $\bar{I}_i$ a matriz de inércia em $G_i$ e $J(G_i,q)$ a matriz jacobiana cinemática do centro de gravidade do i-ésimo elemento. 


Como todas as juntas que compõem o conjunto braço robótico-bicóptero são de revolução, então $J(G_i,q)$ \cite{Park2017} vale 

\begin{equation}
    J(G_i)=
    \begin{bmatrix}
    z_{i-1}\times \overrightarrow{O_{i-1} O_{G_i}}\\
    z_{i-1}
    \end{bmatrix}
    \label{eq:jicgi}
\end{equation}

In [31]:
# Build the Di matriz
def Di_M(Jc_gi,mI):
    M = (Jc_gi.T)*mI*Jc_gi;
    return M

In [32]:
D1 = Di_M(Jc1,mI_1)
D2 = trigsimp(Di_M(Jc2,mI_2))
D3 = trigsimp(Di_M(Jc3,mI_3))
D4 = trigsimp(Di_M(Jc4,mI_4))
DP = trigsimp(Di_M(Jcp,mI_p))
Dtotal = D1+D2+D3+D4+DP

In [33]:
D1

Matrix([
[Iz_1, 0, 0, 0, 0],
[   0, 0, 0, 0, 0],
[   0, 0, 0, 0, 0],
[   0, 0, 0, 0, 0],
[   0, 0, 0, 0, 0]])

In [34]:
 trigsimp(D2).subs({theta1:0,theta2:0,theta4:0})

Matrix([
[Iz_2,    0, 0, 0, 0],
[   0, Iy_2, 0, 0, 0],
[   0,    0, 0, 0, 0],
[   0,    0, 0, 0, 0],
[   0,    0, 0, 0, 0]])

In [35]:
trigsimp(D3).subs({theta1:0,theta2:0,theta4:0})

Matrix([
[Iz_3 + l2**2*m_3,                                 0,                     0, 0, 0],
[               0, Iy_3 + l2**2*m_3 + l_{G_3}**2*m_3, Iy_3 + l_{G_3}**2*m_3, 0, 0],
[               0,             Iy_3 + l_{G_3}**2*m_3, Iy_3 + l_{G_3}**2*m_3, 0, 0],
[               0,                                 0,                     0, 0, 0],
[               0,                                 0,                     0, 0, 0]])

In [36]:
trigsimp(D4).subs({theta1:0,theta2:0,theta4:0})

Matrix([
[Iz_4 + l2**2*m_4,                            0,                0,    0, 0],
[               0, Iy_4 + l2**2*m_4 + l3**2*m_4, Iy_4 + l3**2*m_4,    0, 0],
[               0,             Iy_4 + l3**2*m_4, Iy_4 + l3**2*m_4,    0, 0],
[               0,                            0,                0, Ix_4, 0],
[               0,                            0,                0,    0, 0]])

In [37]:
trigsimp(DP).subs({theta1:0,theta2:0,theta4:0})

Matrix([
[Iz_p + l2**2*m_p,                                                      0,                                          0,        l2*l4*m_p,    0],
[               0, Iy_p + l2**2*m_p + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p, Iy_p + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p,                0,    0],
[               0,             Iy_p + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p, Iy_p + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p,                0,    0],
[       l2*l4*m_p,                                                      0,                                          0, Ix_p + l4**2*m_p, Ix_p],
[               0,                                                      0,                                          0,             Ix_p, Ix_p]])

In [38]:
Dtotal.subs({theta1:0,theta2:0,theta4:0})

Matrix([
[Iz_1 + Iz_2 + Iz_3 + Iz_4 + Iz_p + l2**2*m_3 + l2**2*m_4 + l2**2*m_p,                                                                                                                                0,                                                                                     0,               l2*l4*m_p,    0],
[                                                                   0, Iy_2 + Iy_3 + Iy_4 + Iy_p + l2**2*m_3 + l2**2*m_4 + l2**2*m_p + l3**2*m_4 + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p + l_{G_3}**2*m_3, Iy_3 + Iy_4 + Iy_p + l3**2*m_4 + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p + l_{G_3}**2*m_3,                       0,    0],
[                                                                   0,                                            Iy_3 + Iy_4 + Iy_p + l3**2*m_4 + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p + l_{G_3}**2*m_3, Iy_3 + Iy_4 + Iy_p + l3**2*m_4 + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p + l_{G_3}**2*m_3,                       0,    0],
[                              

In [39]:
K = 1/2* q_dot.T * Dtotal * q_dot

<h2> Calculo da Energia Potencial</h2>

A energia potencial do segmento i é definida por:

\begin{equation}
    \mathcal{P}_i = m_i g \vec{z}_0 \cdot \overrightarrow{O_0G_i}(q)_{/R_0} = m_i g z_{G_i/R_0}
    \label{eq:PotencialEnergy}
\end{equation}

em que $m_i$ e $G_i$ são a massa e o centro de gravidade do $i$-ésimo segmento, $g$ a constante universal da gravitação .

A energia total do robô pode ser escrita por:

\begin{equation}
    \mathcal{P}(q) = \sum_{i=1}^n \mathcal{P}_i(q)
    \label{eq:PotencialEnergy_sum}
\end{equation}


In [40]:
# definindo os vetores até os centro de gravidade como vetores de \mathbb{R}^4
#perceba que og2 esta na origem de R_1 
#perceba que og3 esta definido a partir do referencial R_2 por isso usa-se T02
#perceba que og4 esta na origem de R_3 
#perceba que ogp esta na origem de R_p
cg_origem={lg2:0,lg4:0}
og1= Matrix([0,0,lg1,1])
og2=T01*Matrix([lg2,0,0,1]).subs(cg_origem)
og3=T02*Matrix([0,-lg3,0,1]).subs(cg_origem)
og4=T03*Matrix([lg4,0,0,1]).subs(cg_origem)
ogp=T0P*Matrix([0,0,0,1]).subs(cg_origem)

In [41]:
#og1.subs({theta1:0,theta2:0,theta4:0})
#og2.subs({theta1:0,theta2:0,theta4:0})
#og3.subs({theta1:0,theta2:0,theta4:0})
#og4.subs({theta1:0,theta2:0,theta4:0})
#ogp.subs({theta1:0,theta2:0,theta4:0})

In [42]:
# ogp= og4
P1 = m1* g * Matrix([0,0,1,0]).T * og1
P2 = m2* g * Matrix([0,0,1,0]).T * og2
P3 = m3* g * Matrix([0,0,1,0]).T * og3
P4 = m4* g * Matrix([0,0,1,0]).T * og4
PP = mp* g * Matrix([0,0,1,0]).T * ogp

In [43]:
Ptotal = P1+ P2 + P3 + P4 + PP
#Ptotal

<h2> Calculo do Lagrangiano </h2>

Após estabelecer as energias cinéticas e potenciais do robô, pode-se definir o Lagrangiano:

\begin{equation}
    \mathcal{L}  = \mathcal{K} - \mathcal{P}(\textbf{q})
    \label{eq:Lagrangiano_f}
\end{equation}


In [44]:
L= K-Ptotal

<h2> Calculo dos termos da equaçoes de Euler Lagrange </h2>

A partir da formulação das equações de Euler-Lagrange o modelo dinâmico de braço robóticos é definido por 

\begin{equation}
     D(q)\ddot{q}+C(q,\ddot{q})+G(q)=\Gamma
     \label{eq:EulerLagrange}
\end{equation} 

em que o termo $D(q)\ddot{q}$ designa as contribuições inerciais, $C(q,\dot{q})$ corresponde as contribuições centrífugas e de Coriolis, $G(q)$ representa as contribuições da gravidade, $\Gamma$ diz respeito aos esforços articulares e $q$, $\dot{q}$ e $\ddot{q}$ são as posições, velocidades  e as acelerações das juntas que compõem o conjunto, respectivamente.

Matematicamente o termo $D(q)$ é definido por

\begin{equation}
    D(q)= \sum_{i=1}^5[J_i(G_i,q)]^T\cdot\begin{bmatrix}
    m_i & 0 & 0 & \emptyset_{1x3}\\
    0 & m_i & 0 & \emptyset_{1x3}\\
    0 & 0 & m_i & \emptyset_{1x3}\\
    \emptyset_{3x3} & \emptyset_{3x3} & \emptyset_{3x3} & \bar{I}_i
    \end{bmatrix}\cdot J_i(G_i,q)
     \label{eq:D_euler_lagrange}
\end{equation}

sendo que $G_i$ o centro de gravidade do elemento $i$, $\emptyset_{3x3}$ uma matriz de zeros, $\emptyset_{1x3}$ um vetor linha de zeros, $\bar{I}_i$ a matriz de inércia em $G_i$ e $J_i(G_i,q)$ a i-ésima coluna da matriz jacobiana cinemática do centro de gravidade do i-ésimo elemento. 

Como todas as juntas que compõem o conjunto braço robótico-bicóptero são de revolução, então $J_i(G_i,q)$ \cite{Park2017} vale 

\begin{equation}
    J_i(G_i)=
    \begin{bmatrix}
    z_{i-1}\times \overrightarrow{O_{i-1} O_{G_i}}\\
    z_{i-1}
    \end{bmatrix}
    \label{eq:jicgi}
\end{equation}

O termo $C(q,\dot{q})$ é definido por

\begin{equation}
     C(q, \dot{q})=\left( \sum_{k=1}^4 \frac{\partial D(q) }{\partial q_k}\dot{q}_k \right) \dot{q} - \begin{bmatrix}
    \frac{1}{2}\dot{q}^T\frac{\partial D(q)}{\partial q_1\dot{q}}\\
    \vdots\\
    \frac{1}{2}\dot{q}^T\frac{\partial D(q)}{\partial q_4\dot{q}}\\
    0
    \end{bmatrix}
    \label{eq:C_euler_lagrange}
\end{equation}

E o termo $G(q)$ vale

\begin{equation}
    G(q)= \sum_{i=1}^5 m_i g  \begin{bmatrix}
    \frac{\partial zG_{i/R0}}{\partial q_1}\\
    \vdots\\
     \frac{\partial zG_{i/R0}}{\partial q_4}\\
     0
    \end{bmatrix}
    \label{eq:Q_euler_lagrange}
\end{equation}


In [45]:
# Termo D(q) contribuicao inercial vale D_total ja calculado precedemente
#Dtotal
#q[1]
#diff(Dtotal,q[1])*q_dot[1]

<h3> Criação de $C(q)$</h3>

In [46]:
def C_t(q,q_dot,D_total):
    C_1 = zeros(5,5)
    C_2 = zeros(5,1)

    for n in range(4):
        # Part 1 = dD(q)/dt = sum (partial D(q)/partial qi)q_dot
        C_1= C_1 + (diff(D_total,q[n])*q_dot[n])

    C_1 = C_1*q_dot

    for n in range(4):
        # Part 2 = column matrix (1/2)*(q_dot)^T.(partial D(q)/partial qi).q_dot
        C_2[n] = (1/2)*((q_dot.T)*diff(D_total,q[n])*q_dot)

    C = C_1 - C_2;
    return C

In [47]:
# Termo C(q,qdot) contribuicao coriolis
Ctotal = C_t(q, q_dot, Dtotal)
#Ctotal

<h3> Criação de $G(q)$</h3>

In [48]:
def G_t(q,P_total):
    G = zeros(5,1)
    for n in range(4):
        G[n] = diff(P_total,q[n])
    return G

In [49]:
# Termo G(q)  vale as contribuicoes da gravidade 
Gtotal = G_t(q,Ptotal)
Gtotal

Matrix([
[                                                                                         0],
[(l2*cos(theta2) + l_{G_3}*sin(theta2))*g*m_3 + g*l2*m_4*cos(theta2) + g*l2*m_p*cos(theta2)],
[                                                                                         0],
[                                                                      g*l4*m_p*sin(theta4)],
[                                                                                         0]])

O termo $\Gamma$ é vetor resultante da interação entre os momentos dos acionadores do braço robótico $\Gamma_m$, do contrapeso $\Gamma_c$, dos atritos rotacionais e da forças aplicadas sobre o ambiente $\Gamma_e$.  Isto é,

\begin{equation}
    \Gamma = \Gamma_m + \Gamma_c - \Gamma_e -\Gamma_f
    \label{eq:Robot_Efforts}
\end{equation}

o do sistema construído, não há acionadores nas juntas do braço da base. Logo, $\Gamma_m = \emptyset_{4\times6}$.

<h1> Esforços devido ao atrito viscoso $\Gamma_f$ </h1>

Os coeficientes de atrito viscoso em torno dos ângulos de rotação $\theta_2$ e $\theta_4$ foram determinados seguindo o procedimento descrito no Apêndice.  Já o atrito em torno de $\theta_1$ foi desconsiderado, pois não foi viável determiná-lo experimentalmente de forma segura. Logo $\Gamma_f$ é descrito por

\begin{equation}
    \Gamma_f= \begin{bmatrix}
    0\\
    f_{s_2}\dot{\theta}_2\\
    0\\
    f_{s_4}\dot{\theta}_4\\
    0
    \end{bmatrix}
    \label{eq:gamma_f}
\end{equation}



In [50]:
Gamma_f = diag(fs1, fs2, fs3, fs4,fs5)*q_dot
Gamma_f = Gamma_f.subs({fs1:0,fs3:0,fs5:0})
Gamma_f

Matrix([
[          0],
[fs2*theta2'],
[          0],
[fs4*theta4'],
[          0]])

<h1> Esforços realizados pelo contrapeso $\Gamma_c$ </h1>


Os esforços realizados pelo contrapeso $\Gamma_c$ em um braço robótico constituído por $n$ elementos pode ser calculado da seguinte maneira:

\begin{equation}
    \Gamma_c=\sum^n_{i=1}\Gamma_m,i=\begin{bmatrix} 
    m_1g\frac{\partial zG_{1/R0}}{\partial q_1} & + & m_2g\frac{\partial zG_{2/R0}}{\partial q_1} & + & ... & + & m_ng\frac{\partial zG_{n/R0}}{\partial q_1}\\
    & & m_2g\frac{\partial zG_{2/R0}}{\partial q_2} & + & ... & + & m_ng\frac{\partial zG_{n/R0}}{\partial q_2}\\
    & & & & & \ddots & \vdots\\
    & & & & & & m_ng\frac{\partial zG_{n/R0}}{\partial q_n}
     \end{bmatrix}
    \label{eq:tensorpesanteur_formula}
\end{equation}


Para o conjunto do braço robótico-bicóptero o contrapeso é apenas utilizado para contrabalancear os esforços devido as forças peso dos elementos do braço robótico, isto é até o elemento 4, e não ao esforço devido a força peso do bicóptero, sendo assim:

\begin{equation}
    \Gamma_c= \sum^4_{i=1}\Gamma_m,i = \begin{bmatrix} 
    0\\
    gl_2c(\theta_2)(m_3+m_4)+gl_{G_3}m_3s(\theta_2)\\
    0\\
    0\\
    0\end{bmatrix}
    \label{eq:tensorpesanteur_final}
\end{equation}

In [51]:
Gamma_c=zeros(5,1)
# Nao realizamos para o peso do elemento final que é o drone 
Gamma_c[0] = diff(P1,q[0])+diff(P2,q[0])+diff(P3,q[0])+diff(P4,q[0])#+diff(PP,q[0])
Gamma_c[1] = diff(P2,q[1])+diff(P3,q[1])+diff(P4,q[1])#+diff(PP,q[1])
Gamma_c[2] = diff(P3,q[2])+diff(P4,q[2])#+diff(PP,q[2])
Gamma_c[3] = diff(P4,q[3])#+diff(PP,q[3])
Gamma_c

Matrix([
[                                                                  0],
[(l2*cos(theta2) + l_{G_3}*sin(theta2))*g*m_3 + g*l2*m_4*cos(theta2)],
[                                                                  0],
[                                                                  0],
[                                                                  0]])

<h1> Calculo de $\Gamma_e$ </h1>

\begin{equation}
    \Gamma_e= \begin{bmatrix}{}^{P}J_n(O_p,\textbf{q}) \end{bmatrix}^T \cdot {}^P\mathcal{T}_e(P)
    \label{eq:Gamma_eP}
\end{equation}

sendo ${}^{P}J_n(O_p,q)$ a n-ésima coluna da matriz jacobiana cinemática definida no referencial $R_P$ do elemento $S_n$ ao ponto de aplicação $O_p$ da força exterior, que no nosso caso corresponde ao ponto origem do referencial do bicóptero, e ${}^P\mathcal{T}_e(O_p)$ o vetor dos esforços aplicadas pelo órgão terminal do robô sobre o ambiente expresso no referencial $R_P$.

\begin{equation}
    {}^P\mathcal{T}_e(O_p) =\begin{bmatrix}
    f_x & f_y & f_z & \tau_x & \tau_y &\tau_z
    \end{bmatrix}^T
    \label{eq:t_e}
\end{equation}

em que $f_x,\ f_y,\ f_z$ correspondem as componentes da força total e $\tau_x,\ \tau_y,\ \tau_z$ as componentes do momento total nos eixos $X_{j_P},\ Y_{j_P}$ e $Z_{j_P}$ respectivamente.

In [52]:
#Forca expressa no referencial P
#tau_e:=<<f[x],f[y],f[z],tau[x],tau[y],tau[z]>>:
tau_eP = Matrix([fx,fy,fz,taux,tauy,tauz])

Ainda que devido as restrições do movimento do braço base o vetor ${}^P\mathcal{T}_e(P))$ vale

\begin{equation}
    {}^P\mathcal{T}_e(P) =\begin{bmatrix}
    0 & f_y & f_z & \tau_x & 0 & 0
    \end{bmatrix}^T
    \label{eq:t_e}
\end{equation}


In [53]:
# nosso aplicacao taux=tauy=fz=0
tau_eP = Matrix([0,fy,fz,tauz,0,0])

até então não temos $\begin{bmatrix}{}^{P}J_n(O_p,\textbf{q}) \end{bmatrix}^T$ apenas $\begin{bmatrix}{}^{0}J_n(O_p,\textbf{q}) \end{bmatrix}^T$ devemos buscar realizar a chamada mudança de base da matriz jacobiana cinemática

Para representar o vetor esforco em outro referencial é necessario as formulações de vetor em $SO_3$ representação adjunta da matriz homogênea e inversa da matriz homogênea

Para entender indica-se o __[exemplo da maçã](https://modernrobotics.northwestern.edu/nu-gm-book-resource/3-4-wrenches/#department)__

In [54]:
#representacao de um vetor em so3
def v_2so3(v):
    p = zeros(3)
    p[0,1] = -v[2,0]
    p[0,2] = v[1,0]
    p[1,2] = -v[0,0]
    p[1,0] = v[2,0]
    p[2,0] = -v[1,0]
    p[2,1] = v[0,0]
    return p

#representacao adjunta da matriz Transformacao homogenea
def T_adj(Thom):
    p = v_2so3(Thom[0:3,3])

    K = p*rt_M(Thom)
    M = zeros(6)

    M[0:3,0:3] = Thom[0:3,0:3]
    M[3:6,0:6] = Matrix([K[0:3,0:3]]).col_insert(3,Thom[0:3,0:3]) 

    return M

#Inversa da matriz homogênea
def inv_Thom(Thom):
    Ka = (rt_M(Thom)).T
    Ke = -Ka*Thom[0:3,3] 

    M = Ka[0:3,0:3]
    M = M.col_insert(3,Ke[0:3,0])

    M = M.row_insert(4, Matrix([0,0,0,1]).T)
    return M

In [55]:
#Calcula-se a matrix inversa (T_0_P) da matriz transformacao homogenea T_0P 
TP0 =inv_Thom(T0P)

#Passa-se o vetor 3x1 p da entao matriz inversa para s03
p=v_2so3(TP0[0:3,3])

#Finalmente calcula-se a matrix adjunta de T_P_0
MAdjTP0 = T_adj(TP0)

#TP0
#p
#MAdjTP0

#Constroi-se uma matrix capaz de colocar os elementos momentos nas ultimas 3 posicoes do vetor 6x1 e colocar as forcas nas 3 primeiras posicoes do vetor
MF_M = zeros(6)
MF_M[0,3] =1
MF_M[1,4] =1
MF_M[2,5] =1
MF_M[3,0] =1
MF_M[4,1] =1
MF_M[5,2] =1

<h2>Mudança de base da matriz jacobiana para o referencial $R_P$</h2>
    
\begin{equation}
     {}^{P}J_n(O_p,q)= \begin{bmatrix}
     {}^{P}M_0 & \emptyset_{3x3}  \\
     \emptyset_{3x3} & {}^{P}M_0\end{bmatrix} \cdot {}^{0}J_n(O_p,q)
    \label{eq:mudança de base_Jcp}
\end{equation}

em que ${}^{P}M_0$ corresponde à matriz de rotação que compõe a matriz de transformação homogênea ${}^{P}T_0= ({}^{0}T_P)^{-1}$.

In [56]:
#Aqui mostra-se como os esforcos em P sao traduzidos no referencial 0
PJP = trigsimp(ch_M(rt_M(TP0))*JcPP)
PJP.T

Matrix([
[                      l4*sin(theta4), l2*cos(theta2)*cos(theta4),  l2*sin(theta4)*cos(theta2),  0,  sin(theta4), -cos(theta4)],
[l2*sin(theta2) - l3 - l4*cos(theta4), l2*sin(theta4)*cos(theta2), -l2*cos(theta2)*cos(theta4),  0, -cos(theta4), -sin(theta4)],
[                -l3 - l4*cos(theta4),                          0,                           0,  0, -cos(theta4), -sin(theta4)],
[                                   0,                         l4,                           0, -1,            0,            0],
[                                   0,                          0,                           0, -1,            0,            0]])

<h3>Calculo de $\Gamma_e$ (sem mecanismo de tilt)</h3>

In [57]:
#Calcula-se Gamma_e
Gamma_e = PJP.T*(tau_eP)

<h3>Calculo de $\Gamma_b$ (sem mecanismo de tilt)</h3>

Diferentemente de aplicações usuais em robótica onde os esforços são realizados pelo robô sobre o ambiente, o modelo do conjunto braço robótico-bicóptero exige o termo que traduz os esforços do ambiente sobre o conjunto. Esse termo é representado por $\Gamma_b$ e é definido pela relação:

\begin{equation}
    \Gamma_b=-\Gamma_e
    \label{eq:relacao_gammab_gammae}
\end{equation}

In [58]:
#Nosso caso queremos Gamma_b nao coloquei o negativo mas mais abaixo ele aparece
Gamma_b = trigsimp(Gamma_e)
Gamma_b

Matrix([
[(f_y*cos(theta4) + f_z*sin(theta4))*l2*cos(theta2)],
[(f_y*sin(theta4) - f_z*cos(theta4))*l2*cos(theta2)],
[                                                 0],
[                                    f_y*l4 - tau_z],
[                                            -tau_z]])

<h2> Traduzindo as variaveis de entrada com o mecanismo de vetorização</h2>

<div>
<img src="img/forcas_bicoptero.png" width="500">
</div>

\begin{equation}
   \begin{bmatrix}
    0\\
    f_y\\
    f_z\\
    \tau_x\\
    0 \\
    0 \end{bmatrix} =
    \begin{bmatrix}
    0 & 0 \\
    s(\alpha_1) & -s(\alpha_2)\\
    -c(\alpha_1) & -c(\alpha_2)\\
    -b c(\alpha_1) & b c(\alpha_2)\\
    0 & 0 \\
    0 & 0 
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
    F_{e_1}\\
    F_{e_2}
    \end{bmatrix}
    \label{eq:vetorizacao}
\end{equation}

In [59]:
tilt = Matrix([zeros(1,2),[sin(alpha1), -sin(alpha2)],[-cos(alpha1), -cos(alpha2)],[-b1*cos(alpha1), b2*cos(alpha2)], zeros(2,2)])
forces= Matrix([f1,f2])

In [60]:
tilt

Matrix([
[               0,               0],
[     sin(alpha1),    -sin(alpha2)],
[    -cos(alpha1),    -cos(alpha2)],
[-b_1*cos(alpha1), b_2*cos(alpha2)],
[               0,               0],
[               0,               0]])

<h3>Calculo de $\Gamma_e$ com vetorização da força empuxo</h3>

In [61]:
#Calcula-se Gamma_e
Gamma_e = PJP.T*tilt*forces

<h3>Calculo de $\Gamma_b$ </h3>

In [62]:
#Nosso caso queremos Gamma_b ainda não foi colocado o negativo mas será feito na equacao total do movimento
Gamma_b = trigsimp(Gamma_e)

In [63]:
#Visualização das forças no estado zero zero zero zero zero zero
Gamma_b.subs({theta1:0,theta2:0,theta4:0,alpha1:0,alpha2:0})

Matrix([
[                0],
[   (F_1 + F_2)*l2],
[                0],
[F_1*b_1 - F_2*b_2],
[F_1*b_1 - F_2*b_2]])

<h2> Equação Total do Movimento </h2>

\begin{equation}
    \emptyset_{4x6} =D(q)\ddot{q} + C(q,\dot{q}) + G(q) + \Gamma_f - \Gamma_c - \Gamma_b 
    \label{eq:modele_dyn_fb}
\end{equation}


In [64]:
Mdf=Dtotal*q_ddot+Ctotal+Gtotal+Gamma_f-Gamma_b-Gamma_c

<h2>Iniciando Equacao Solução</h2>

A equação relativa a $\ddot{\theta_3}$ é completamente ignorada, Eq tem dimensão igual a $3\times1$ devido a relação entre $\theta_3$ e $\theta_2$

In [65]:
Eq=zeros(3,1)

In [66]:
#Isolando as variaveis q_ddot
Eq[0]=solve(Mdf[0], q_ddot[0])  #Relativo a Theta dois pontos 1
Eq[1]=solve(Mdf[1], q_ddot[1])  #Relativo a Theta dois pontos 2
#Eq[2]=solve(Mdf[2], q_ddot[2])  #Relativo a Theta dois pontos 3
Eq[2]=solve(Mdf[3], q_ddot[3])  #Relativo a Theta dois pontos 4


In [67]:
#%store Eq

A fim de aplicar uma estratégia de controle linear é necessário obter a partir do modelo completo apresentado anteriormente o modelo linearizado, esse processo de linearização é realizado na seção que segue.

<h2> Linearização (Método de Newton) </h2>

$\let\hdots\ldots$

$\newcommand{\parcial}[2]{\frac{\partial #1}{\partial #2}\vert_{\vec{x}_0,\vec{u}_0}}$
$\newcommand{\lineparcial}[2]{\parcial{#1}{#2_1} & \parcial{#1}{#2_2} & \parcial{#1}{#2_3} & \hdots & \parcial{#1}{#2_6}}$
$\newcommand{\lineparcialB}[2]{\parcial{#1}{#2_1} & \parcial{#1}{#2_2} & \parcial{#1}{#2_3} & \parcial{#1}{#2_4}}$
$\newcommand{\lineA}{\vdots & \vdots & \vdots & \ddots & \vdots}$
$\newcommand{\lineB}{\vdots & \vdots & \vdots & \vdots}$

\begin{align}
    A &= \begin{bmatrix}
    \lineparcial{f_1}{x}\\
    \lineparcial{f_2}{x}\\
    \lineA\\
    \lineparcial{f_6}{x}
    \end{bmatrix}&,\\
    B &= \begin{bmatrix}
    \lineparcialB{f_1}{u}\\
    \lineparcialB{f_2}{u}\\
    \lineB\\
    \lineparcialB{f_6}{u}
    \end{bmatrix}&,\\
    C &= \begin{bmatrix}
    \lineparcial{g_1}{x}\\
    \lineparcial{g_2}{x}\\
    \lineA\\
    \lineparcial{g_6}{x}
    \end{bmatrix} &\text{e}\\
    D &= \begin{bmatrix}
    \lineparcialB{g_1}{u}\\
    \lineparcialB{g_2}{u}\\
    \lineB\\
    \lineparcialB{g_6}{u}
    \end{bmatrix}
\end{align}

sendo que $\vec{x}_0$ e $\vec{u}_0$ são vetores constantes das variáveis de entradas e saída no ponto de equilíbrio. Já $f$ e $g$ são definidas por

\begin{align}
    f(\vec{x},\vec{u})=\begin{cases}
    f_1 = \dot{x}_1 = x_4\\
    f_2 = \dot{x}_2 = x_5\\
    f_3 = \dot{x}_3 = x_6\\
    f_4 = \ddot{x}_1 = \ddot{q}_1 \\
    f_5 = \ddot{x}_2 = \ddot{q}_2 \\
    f_6 = \ddot{x}_3 = \ddot{q}_4 \\
    \end{cases} && \hfill\text{e} && 
    g(\vec{x})=\begin{cases}
    g_1 = x_1 \\
    g_2 = x_2 \\
    g_3 = x_3 \\
    \hfill \vdots \\
    g_6 = x_6 \\
    \end{cases} 
\end{align}


<h2>Iniciando o vetor Estado e Comando</h2>
Como o vetor de estados vale

$\begin{equation} \vec{x}=\begin{bmatrix} 
\theta_{1}\\
\theta_{2}\\
\theta_{4}\\
\dot{\theta}_{1}\\
\dot{\theta}_{2}\\
\dot{\theta}_{4}\end{bmatrix}
\end{equation}$

O vetor dos comandos vale

$\begin{equation}\vec{u}= \begin{bmatrix}
F_1\\
F_2\\
\alpha_1\\
\alpha_2
\end{bmatrix}
\end{equation}$



In [68]:
#states=Matrix([theta1, theta2, theta4, dot theta1 dottheta2 dottheta4])
states=q.row_insert(4, q_dot)
states.row_del(6)
states.row_del(2)
states.row_del(6)
states.row_del(6)
states

Matrix([
[ theta1],
[ theta2],
[ theta4],
[theta1'],
[theta2'],
[theta4']])

In [69]:
inputs=Matrix([f1,f2,alpha1,alpha2])
inputs

Matrix([
[   F_1],
[   F_2],
[alpha1],
[alpha2]])

In [70]:
len(inputs)#4
len(states)#6
len(Eq)#3

3

<h2>Iniciando o vetor estado inicial</h2>

In [71]:
trim_inputs={f1:f10,f2:f20,alpha1:alpha10,alpha2:alpha20}

trim_position={theta1:theta10,theta2:theta20,theta3:theta30,theta4:theta40}
trim_speed={diff(theta10,t):omega10,diff(theta20,t):omega20,diff(theta30,t):omega30,diff(theta40,t):omega40}
trim_acel={diff(diff(theta10,t),t):psi10,diff(diff(theta20,t),t):psi20,diff(diff(theta30,t),t):psi30,diff(diff(theta40,t),t):psi40}

#Estados iniciais
val_pos={theta10: 0.00,theta20: 0.00,theta30: -(pi/2),theta40: 0.00}
val_speed={omega10: 0.00,omega20: 0.00,omega30: 0.00,omega40: 0.00}
val_acel={psi10: 0.00,psi20: 0.00,psi30: 0.00,psi40: 0.00}
val_inputs={alpha10: 0.00,alpha20: 0.00} #f10: g*m4/2,f20: g*m4/2}

In [72]:
Eq_trim=zeros(len(Eq),1)

In [73]:
for i in range (len(Eq)):
#    Eq_trim[i]=Eq[i].subs(Trim).subs(states_value).subs(inputs_value)
    Eq_trim[i]=Eq[i].subs(trim_inputs).subs(val_inputs).subs(trim_position).subs(trim_acel).subs(trim_speed).subs(val_pos).subs(val_acel).subs(val_speed).factor().nsimplify()

In [74]:
Eq_trim
#Observe que se b1=b2 e f10=f20=(1/2)g*m4 tudo é todas as acelerações são iguais a zero o que significa que está em equilibrio

Matrix([
[                                                                                                                                                                                  0],
[(F_10 + F_20 - g*m_p)*l2/(2*(Iy_2/2 + Iy_3/2 + Iy_4/2 + Iy_p/2 + l2**2*m_3/2 + l2**2*m_4/2 + l2**2*m_p/2 + l3**2*m_4/2 + l3**2*m_p/2 + l3*l4*m_p + l4**2*m_p/2 + l_{G_3}**2*m_3/2))],
[                                                                                                                                    (F_10*b_1 - F_20*b_2)/(Ix_4 + Ix_p + l4**2*m_p)]])

In [75]:
#Eq

<h1>Dimensionamento correto das matrizes A e B</h1>

a matrix $A$ deve ser de dimensão $6\times6$ e $B$ de dimensão $4\times6$

sendo que as primeiras 3 linhas da matriz A são

$\begin{equation}
\begin{bmatrix}
0 & 0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0\\
0& 0 & 0 & 0 & 0 & 1\\
\end{bmatrix}\end{equation}$

e as primeiras 3 linhas da matriz B são

$\begin{equation}
\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0& 0 & 0 & 0 \\
\end{bmatrix}\end{equation}$


In [76]:
A=zeros(3,3).col_insert(3, eye(3))
A=A.row_insert(4, zeros(3,6))
A

Matrix([
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0]])

In [77]:
B = zeros(6, 4) 
inputs_value={alpha10: 0,alpha20: 0} #f10: g*m4/2,f20: g*m4/2}
B

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [78]:
for i in range(len(Eq)):
    for j in range(len(states)):
         A[i+3,j]=( diff( Eq[i], states[j] ) ).subs(trim_inputs).subs(val_inputs).subs(trim_position).subs(trim_acel).subs(trim_speed).subs(val_pos).subs(val_acel).subs(val_speed).factor().nsimplify()


In [79]:
for i in range(len(Eq)):
    for j in range(len(inputs)):
        B[i+3,j]= ( diff( Eq[i], inputs[j] ) ).subs(trim_inputs).subs(val_inputs).subs(trim_position).subs(trim_acel).subs(trim_speed).subs(val_pos).subs(val_acel).subs(val_speed).factor().nsimplify()

In [80]:
A

Matrix([
[0,                                                                                                                                                                                                                    0,                                                                                        0, 1,                                                                                                                                                               0,                              0],
[0,                                                                                                                                                                                                                    0,                                                                                        0, 0,                                                                                                                                                               1,               

In [81]:
B

Matrix([
[                                                                                                                                                            0,                                                                                                                                                             0,                                                                              0,                                                                               0],
[                                                                                                                                                            0,                                                                                                                                                             0,                                                                              0,                                                                               0],
[                            

In [82]:
#delta_states=Matrix([(theta1-theta10), (theta2-theta20), (theta3-theta30), (theta4-theta40)])
#delta_inputs=Matrix([(alpha1-alpha10), (alpha2-alpha20), (f1-f10), (f2-f20)])
#Sol=A*delta_states+B*delta_inputs
#Sol

<h2>Particularizando</h2>
<h2>Caso 1</h2>
Seja

$\begin{align}
F_{1_0}&=F_{2_0}=\frac{1}{2}m_pg
\end{align}$


In [127]:
Asimp1=A
Bsimp1=B

In [128]:
Asimp1

Matrix([
[0,                                                                                                                                                                                                                    0,                                                                                        0, 1,                                                                                                                                       0,                              0],
[0,                                                                                                                                                                                                                    0,                                                                                        0, 0,                                                                                                                                       1,                              0],
[0,                          

In [129]:
Bsimp1

Matrix([
[                                                                                                                                    0,                                                                                                                                     0,                                                                              0,                                                                               0],
[                                                                                                                                    0,                                                                                                                                     0,                                                                              0,                                                                               0],
[                                                                                                                            

In [130]:
n,denominator1 = fraction(Asimp1[5,5])
n,denominator2 = fraction(Asimp1[3,2])
n,denominator3 = fraction(Bsimp1[4,1])
numerator1,denominator4 = fraction(Asimp1[4,1])
termos=Matrix([[numerator1],[denominator1], [denominator2], [denominator3],[denominator4]]);
#(2*pow(((0.5*denominator3).nsimplify()),2))=den4
termos

Matrix([
[                                                                                                     (l3*m_4 + l3*m_p + l4*m_p)*(F_10 + F_20 - g*m_p)*l2**2],
[                                                                                                                                    Ix_4 + Ix_p + l4**2*m_p],
[                                                                                       Iz_1 + Iz_2 + Iz_3 + Iz_4 + Iz_p + l2**2*m_3 + l2**2*m_4 + l2**2*m_p],
[                           Iy_2 + Iy_3 + Iy_4 + Iy_p + l2**2*m_3 + l2**2*m_4 + l2**2*m_p + l3**2*m_4 + l3**2*m_p + 2*l3*l4*m_p + l4**2*m_p + l_{G_3}**2*m_3],
[2*(Iy_2/2 + Iy_3/2 + Iy_4/2 + Iy_p/2 + l2**2*m_3/2 + l2**2*m_4/2 + l2**2*m_p/2 + l3**2*m_4/2 + l3**2*m_p/2 + l3*l4*m_p + l4**2*m_p/2 + l_{G_3}**2*m_3/2)**2]])

In [131]:
termos[2].collect(l2)

(m_3 + m_4 + m_p)*l2**2 + Iz_1 + Iz_2 + Iz_3 + Iz_4 + Iz_p

In [132]:
((termos[3].collect(l2)).collect(mp))

(l3**2 + 2*l3*l4 + l4**2)*m_p + (m_3 + m_4 + m_p)*l2**2 + Iy_2 + Iy_3 + Iy_4 + Iy_p + l3**2*m_4 + l_{G_3}**2*m_3

In [152]:
(termos[3].collect(l2).coeff(mp)).factor()

(l3 + l4)**2

Simplificando alguns denominadores

In [133]:
n,den = fraction(Asimp1[4,4])
Asimp1[4,4]=n/den

In [134]:
n,den = fraction(Bsimp1[4,0])
Bsimp1[4,0]=n/den
n,den = fraction(Bsimp1[4,1])
Bsimp1[4,1]=n/den

E simplificando a notação

$\begin{align}
    num_1&= - \left(F_{10} + F_{20} - g m_{p}\right) \left(l_{1} (m_{2} +  m_{3}) - l_{3} ( m_{3} + m_{4} +m_{p} ) - l_{4}( m_{4}+m_{p})\right) l^{2}_{2} \\
    den_1 & = Ix_{4}+Ix_{p}+l^{2}_{4}m_{p} \\
    den_2 & = Iz_{1} + Iz_{2} + Iz_{3} + Iz_{4}+ Iz_{p} + l^{2}_{2} (m_{3} + m_{4}+ m_{p}) \\
    den_3 & = Iy_{2} + Iy_{3} + Iy_{4} + Iy_{p} +  l^{2}_{2} (m_{3} + m_{4}+m_{p}) + l^{2}_{3} m_{4} +l^{2}_{G_3}m_{3} + (l^{2}_{3}+l^{2}_{4})^2 m_{p}  \\
    den_4 & = 2 \left( \frac{1}{2} (Iy_{2}+ Iy_{3} + Iy_{4} + Iy_{p}) + \frac{l^{2}_{2}}{2} ( m_{3} + m_{4} + m_{p} ) + \frac{l^{2}_{3} m_{4}}{2} + \frac{l^{2}_{G_3}m_{3}}{2} + \frac{(l^{2}_{3} + l^{2}_{4})}{2} m_{p} \right)^{2} \\& = 2(\frac{1}{2}den_3)^2
\end{align}$

In [135]:
simpl2=({numerator1:num1,denominator1:den1,denominator2:den2,denominator3:den3,denominator4:den4})
Asimp1=Asimp1.subs(simpl2)
Bsimp1=Bsimp1.subs(simpl2)

In [136]:
Asimp1

Matrix([
[0,         0,                      0, 1,         0,         0],
[0,         0,                      0, 0,         1,         0],
[0,         0,                      0, 0,         0,         1],
[0,         0, -(F_10 + F_20)*l2/den2, 0,         0,         0],
[0, num1/den4,                      0, 0, -fs2/den3,         0],
[0,         0,         -g*l4*m_p/den1, 0,         0, -fs4/den1]])

In [137]:
Bsimp1

Matrix([
[       0,         0,            0,             0],
[       0,         0,            0,             0],
[       0,         0,            0,             0],
[       0,         0, F_10*l2/den2, -F_20*l2/den2],
[ l2/den3,   l2/den3,            0,             0],
[b_1/den1, -b_2/den1, F_10*l4/den1, -F_20*l4/den1]])

In [138]:
delta_states=Matrix([(theta1-theta10), (theta2-theta20), (theta4-theta40), diff(theta1), diff(theta2),diff(theta4)])
delta_inputs=Matrix([(f1), (f2),(alpha1), (alpha2)])
Sol=Asimp1*delta_states+Bsimp1*delta_inputs
Sol

Matrix([
[                                                                                                                        theta1'],
[                                                                                                                        theta2'],
[                                                                                                                        theta4'],
[                                         -(F_10 + F_20)*(theta4 - theta4_0)*l2/den2 + F_10*alpha1*l2/den2 - F_20*alpha2*l2/den2],
[                                                   (theta2 - theta2_0)*num1/den4 + F_1*l2/den3 + F_2*l2/den3 - fs2*theta2'/den3],
[-(theta4 - theta4_0)*g*l4*m_p/den1 + F_1*b_1/den1 + F_10*alpha1*l4/den1 - F_2*b_2/den1 - F_20*alpha2*l4/den1 - fs4*theta4'/den1]])

In [94]:
Sol[3]

-(F_10 + F_20)*(theta4 - theta4_0)*l2/den2 + F_10*alpha1*l2/den2 - F_20*alpha2*l2/den2

In [95]:
Sol[4].subs({num1:0,theta20:0,b1:b,b2:b}).simplify().collect(l2)

((F_1 + F_2)*l2 - fs2*theta2')/den3

In [96]:
Sol[5].subs({theta40:0,b1:b,b2:b}).simplify().collect(b)

((F_1 - F_2)*b + F_10*alpha1*l4 - F_20*alpha2*l4 - fs4*theta4' - g*l4*m_p*theta4)/den1

<h2>Salvando variáveis</h2>

In [97]:
#%store Sol

<h2>Caso 2</h2>

Seja a simplificação dada por

$\begin{align}
l_4&=0\\
F_{1_0}&=F_{2_0}=\frac{1}{2}m_4g
\end{align}$


In [98]:
simpl2=({l4:0})   # f10: g*m4/2,f20: g*m4/2}) # forcas de trimagem
Asimp2=A.subs(simpl2)
Bsimp2=B.subs(simpl2)

In [99]:
Asimp2

Matrix([
[0,                                                                                                                                                                                 0,                                                                                        0, 1,                                                                                                             0,                  0],
[0,                                                                                                                                                                                 0,                                                                                        0, 0,                                                                                                             1,                  0],
[0,                                                                                                                                                                            

In [100]:
Bsimp2

Matrix([
[                                                                                                          0,                                                                                                           0,                                                                              0,                                                                               0],
[                                                                                                          0,                                                                                                           0,                                                                              0,                                                                               0],
[                                                                                                          0,                                                                                                           0,           

In [101]:
n,denominator1 = fraction(Asimp2[5,5])
n,denominator2 = fraction(Asimp2[3,2])
n,denominator3 = fraction(Bsimp2[4,1])
numerator1,denominator4 = fraction(Asimp2[4,1])
termos=Matrix([[numerator1],[denominator1], [denominator2], [denominator3],[denominator4]]);
#(2*pow(((0.5*denominator3).nsimplify()),2))=den4
termos

Matrix([
[                                                                                    (l3*m_4 + l3*m_p)*(F_10 + F_20 - g*m_p)*l2**2],
[                                                                                                                      Ix_4 + Ix_p],
[                                                             Iz_1 + Iz_2 + Iz_3 + Iz_4 + Iz_p + l2**2*m_3 + l2**2*m_4 + l2**2*m_p],
[                           Iy_2 + Iy_3 + Iy_4 + Iy_p + l2**2*m_3 + l2**2*m_4 + l2**2*m_p + l3**2*m_4 + l3**2*m_p + l_{G_3}**2*m_3],
[2*(Iy_2/2 + Iy_3/2 + Iy_4/2 + Iy_p/2 + l2**2*m_3/2 + l2**2*m_4/2 + l2**2*m_p/2 + l3**2*m_4/2 + l3**2*m_p/2 + l_{G_3}**2*m_3/2)**2]])

Simplificando alguns denominadores

In [102]:
n,den = fraction(Asimp2[4,4])
Asimp2[4,4]=n/den

In [103]:
n,den = fraction(Bsimp2[4,0])
Bsimp2[4,0]=n/den
n,den = fraction(Bsimp2[4,1])
Bsimp2[4,1]=n/den

E simplificando a notação

$\begin{align}
    num_1&= - \left(F_{10} + F_{20} - g m_{p}\right) \left(l_{1} (m_{2} +  m_{3}) - l_{3} ( m_{3} + m_{4} +m_{p}) \right) l^{2}_{2} \\
    den_1 & = Ix_{p} \\
    den_2 & = Iz_{1} + Iz_{2} + Iz_{3} + Iz_{4}+ Iz_{p} + l^{2}_{2} (m_{2} + m_{3} + m_{4}+ m_{p}) \\
    den_3 & = Iy_{2} + Iy_{3} + Iy_{4} + Iy_{p} + l^{2}_{1} (m_{2} + m_{3}) - 2 l_{1} l_{3} m_{3} + l^{2}_{2} (m_{2} + m_{3} + m_{4}+m_{p}) + l^{2}_{3} (m_{3} + m_{4}+m_{p}) \\
    den_4 & = 2 \left( \frac{1}{2} (Iy_{2}+ Iy_{3} + Iy_{4} + Iy_{p}) + \frac{l^{2}_{1}}{2} (m_{2} + m_{3})  - l_{1} l_{3} m_{3} + \frac{l^{2}_{2}}{2} ( m_{2} +  m_{3} + m_{4} + m_{p} ) + \frac{l^{2}_{3}}{2} (m_{3} + m_{4} + m_{p}) \right)^{2} \\& = 2(\frac{1}{2}den_3)^2
\end{align}$

In [104]:
simpl2=({numerator1:num1,denominator1:den1,denominator2:den2,denominator3:den3,denominator4:den4})
Asimp2=Asimp2.subs(simpl2)
Bsimp2=Bsimp2.subs(simpl2)

In [105]:
Asimp2

Matrix([
[0,         0,                      0, 1,         0,         0],
[0,         0,                      0, 0,         1,         0],
[0,         0,                      0, 0,         0,         1],
[0,         0, -(F_10 + F_20)*l2/den2, 0,         0,         0],
[0, num1/den4,                      0, 0, -fs2/den3,         0],
[0,         0,                      0, 0,         0, -fs4/den1]])

In [106]:
Bsimp2

Matrix([
[       0,         0,            0,             0],
[       0,         0,            0,             0],
[       0,         0,            0,             0],
[       0,         0, F_10*l2/den2, -F_20*l2/den2],
[ l2/den3,   l2/den3,            0,             0],
[b_1/den1, -b_2/den1,            0,             0]])